In [3]:
import re
from time import sleep
from random import randint

from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
prefs = {
    "download.default_directory": "/Users/chrishaun/Desktop/U of T MA/ECO2460 - Text ML",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}

In [7]:
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", prefs)
# For Chrome
driver = webdriver.Chrome(options=options)
# For Firefox
# driver = webdriver.Firefox(executable_path='/path/to/geckodriver')

## Pollievre - Canada First Speech

In [969]:
driver.get("https://www.conservative.ca/canada-first-speech-le-discours-le-canada-dabord/")

In [61]:
item = driver.find_element(By.XPATH, "//p[contains(text(), 'February')]")
feb15pierre = [item.text]

In [67]:
while True:  
    try:
        item = item.find_element(By.XPATH, "following-sibling::p")  # Moves to next sibling
        feb15pierre.append(item.text)
    except:
        break  # Stop if no more siblings exist

In [69]:
feb15_pierre

['February 15, 2025',
 'Bonjour Canada! ',
 'Je sais que des tempêtes de neige ont paralysé Washington récemment, mais ici, dans la capitale du Canada, on ne va jamais laisser quelques flocons de neige gâcher le Jour de notre drapeau national.\n\nI know that snow storms shut down Washington D.C., but here in Canada’s capital, we will never let a few snowflakes ruin our Flag Day. ',
 'On se demande pourquoi Ana a quitté un climat tropical pour des cache-oreilles et une tuque.\n\nWhy would Ana leave a tropical climate for earmuffs and toque? ',
 'Ana and I know that our stories would only be possible here in Canada. Only in this country could a refugee from Venezuela who started in a basement Montreal apartment and the son of a teenage mother raised by school teachers make it to this stage, leading Canada’s founding political party. That is because of the promise this country makes to Canadians of every birthplace and background—that anyone from anywhere can achieve anything. That hard w

## Pollievre - House of Commons Speech

In [9]:
driver.get("https://www.ourcommons.ca/Members/en/pierre-poilievre(25524)#work")

In [11]:
element = driver.find_element(By.XPATH, "//div[contains(@id, 'intervention')]")
element.click()


In [13]:
sleep(1)
element1 = element.find_element(By.XPATH, "//div[contains(@role, 'tabpanel')]")
element2 = element1.find_element(By.XPATH, "./descendant::a")
element2.text
element2.click()

In [15]:
# First find the initial speaker block for Pierre
speaker_pierre = driver.find_element(By.XPATH, "//a[contains(text(), 'Hon. Pierre') and contains(@class, 'PersonSpeakingPopup')]")
main_pierre = speaker_pierre.find_element(By.XPATH, "./parent::b/parent::div/parent::div/parent::span") 
dec16_HOC = [main_pierre.text[61:]]  # Extract first line of speech

next_span = main_pierre.find_element(By.XPATH, "./following::span[@class='paratext'][1]")
dec16_HOC.append(next_span.text)

# Start the loop to find subsequent comments
while True:  
    try:
        # Find the next occurrence of Hon. Pierre
        speaker_pierre = speaker_pierre.find_element(By.XPATH, "./following::a[contains(text(), 'Hon. Pierre') and contains(@class, 'PersonSpeakingPopup')]")
        main_pierre = speaker_pierre.find_element(By.XPATH, "./parent::b/parent::div/parent::div/parent::span") 
        dec16_HOC.append(main_pierre.text)  # Add first speaker block

        current_parent = main_pierre.find_element(By.XPATH, "./parent::*")

        while True:
            try:
                # Find the very next span and div elements
                next_span = main_pierre.find_element(By.XPATH, "./following-sibling::span[1]")
                next_parent = next_span.find_element(By.XPATH, "./parent::*")

                if next_parent != current_parent:
                    break

                # Otherwise, continue extracting the speech text
                dec16_HOC.append(next_span.text)

                # Move to the next span for subsequent iterations
                main_pierre = next_span

            except Exception as e:
                print(f"Error in inner loop: {e}")
                break  # Exit loop if no more elements are found
        
    except Exception as e:
        print(f"Error in outer loop: {e}")
        break  # Exit if no more occurrences of Pierre are found


Error in inner loop: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./following-sibling::span[1]"}
  (Session info: chrome=134.0.6998.45); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104d65804 cxxbridge1$str$ptr + 2785964
1   chromedriver                        0x0000000104d5dddc cxxbridge1$str$ptr + 2754692
2   chromedriver                        0x00000001048b1ea8 cxxbridge1$string$len + 92928
3   chromedriver                        0x00000001048f91d0 cxxbridge1$string$len + 384552
4   chromedriver                        0x00000001048ee878 cxxbridge1$string$len + 341200
5   chromedriver                        0x000000010493a678 cxxbridge1$string$len + 651984
6   chromedriver                        0x00000001048ed35c cxxbridge1$string$len + 335796
7   chromedriver                        0x00

In [17]:
from selenium.webdriver.common.by import By

def scrape_speech(driver, subject, subject_title, day):
    # Dynamically build the XPath for the subject and subject_title
    speaker_element = driver.find_element(By.XPATH, f"//a[contains(text(), '{subject_title}') and contains(@class, 'PersonSpeakingPopup')]")
    main_element = speaker_element.find_element(By.XPATH, "./parent::b/parent::div/parent::div/parent::span") 
    
    # Create a list to store the speech text
    speech_data = [main_element.text]  # Extract first line of speech

    # Get the next span after the speaker's first comment
    next_span = main_element.find_element(By.XPATH, "./following::span[@class='paratext'][1]")
    speech_data.append(next_span.text)

    while True:
        try:
            # Find the next occurrence of the subject
            speaker_element = speaker_element.find_element(By.XPATH, f"./following::a[contains(text(), '{subject_title}') and contains(@class, 'PersonSpeakingPopup')]")
            main_element = speaker_element.find_element(By.XPATH, "./parent::b/parent::div/parent::div/parent::span") 
            speech_data.append(main_element.text)  # Add first speaker block

            current_parent = main_element.find_element(By.XPATH, "./parent::*")

            while True:
                try:
                    # Find the very next span and div elements
                    next_span = main_element.find_element(By.XPATH, "./following-sibling::span[1]")
                    next_parent = next_span.find_element(By.XPATH, "./parent::*")

                    if next_parent != current_parent:
                        break  # If we reach a different parent, stop extracting

                    # Otherwise, continue extracting the speech text
                    speech_data.append(next_span.text)

                    # Move to the next span for subsequent iterations
                    main_element = next_span

                except Exception as e:
                    print(f"Error in inner loop: {e}")
                    break  # Exit loop if no more elements are found

        except Exception as e:
            print(f"Error in outer loop: {e}")
            break  # Exit if no more occurrences of the speaker are found
    
    # Store the speech data into a list named with the specified 'day' (e.g., dec16_HOC)
    globals()[f'{day}_HOC'] = speech_data
    return speech_data


In [19]:
scrape_speech(driver, 'may', 'Hon. Pierre', 'dec16')

Error in inner loop: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./following-sibling::span[1]"}
  (Session info: chrome=134.0.6998.45); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104d65804 cxxbridge1$str$ptr + 2785964
1   chromedriver                        0x0000000104d5dddc cxxbridge1$str$ptr + 2754692
2   chromedriver                        0x00000001048b1ea8 cxxbridge1$string$len + 92928
3   chromedriver                        0x00000001048f91d0 cxxbridge1$string$len + 384552
4   chromedriver                        0x00000001048ee878 cxxbridge1$string$len + 341200
5   chromedriver                        0x000000010493a678 cxxbridge1$string$len + 651984
6   chromedriver                        0x00000001048ed35c cxxbridge1$string$len + 335796
7   chromedriver                        0x00

["Hon. Pierre Poilievre (Leader of the Opposition, CPC):  \n    Mr. Speaker, the Prime Minister has lost control, yet he clings to power. We knew that he had lost control of the borders, immigration, spending, inflation and housing costs, but now he has lost control of his cabinet. The former deputy prime minister and minister of finance resigned just a few hours before presenting Canada's economic update with an enormous deficit.",
 '    When will there be an election?',
 "Hon. Pierre Poilievre (Leader of the Opposition, CPC):  \n     Mr. Speaker, the Prime Minister has lost control, yet he clings to power. He has lost control of immigration; lost control of the borders; lost control of spending, deficits, inflation and housing costs. In fact, his apparently former finance minister was trying to get spending under control by capping the deficit at an already crazy $40 billion, but he took carbon tax Carney's advice and pushed her beyond that guardrail and was trying to throw her off t

In [985]:
dec16_HOC

["Hon. Pierre Poilievre (Leader of the Opposition, CPC):  \n    Mr. Speaker, the Prime Minister has lost control, yet he clings to power. We knew that he had lost control of the borders, immigration, spending, inflation and housing costs, but now he has lost control of his cabinet. The former deputy prime minister and minister of finance resigned just a few hours before presenting Canada's economic update with an enormous deficit.",
 '    When will there be an election?',
 "Hon. Pierre Poilievre (Leader of the Opposition, CPC):  \n     Mr. Speaker, the Prime Minister has lost control, yet he clings to power. He has lost control of immigration; lost control of the borders; lost control of spending, deficits, inflation and housing costs. In fact, his apparently former finance minister was trying to get spending under control by capping the deficit at an already crazy $40 billion, but he took carbon tax Carney's advice and pushed her beyond that guardrail and was trying to throw her off t